In [ ]:
import xarray as xr

from ocr import catalog
from ocr.utils import extract_points

In [ ]:
risk = xr.open_zarr('s3://carbonplan-risks/ocr/v0/intermediates/risk.zarr')

In [ ]:
bbox = (-118.2, 34, -117.5, 34.5)
buildings = catalog.get_dataset('conus-overture-buildings').to_geopandas(
    f"""SELECT * FROM read_parquet('{{s3_path}}') WHERE
bbox.xmin BETWEEN {bbox[0]} AND {bbox[2]} AND
bbox.ymin BETWEEN {bbox[1]} AND {bbox[3]} """
)
addresses = catalog.get_dataset('conus-overture-addresses').to_geopandas(
    f"""SELECT * FROM read_parquet('{{s3_path}}') WHERE
bbox.xmin BETWEEN {bbox[0]} AND {bbox[2]} AND
bbox.ymin BETWEEN {bbox[1]} AND {bbox[3]} """
)

In [ ]:
buildings = buildings[['id', 'geometry', 'bbox', 'height', 'class']]

In [ ]:
addresses = addresses[['id', 'geometry', 'bbox']]

In [ ]:
risk = risk.drop_vars('band')
risk = risk.squeeze(dim='band')

In [ ]:
da = risk['risk']

In [ ]:
buildings['value'] = extract_points(buildings, da)

In [ ]:
addresses['value'] = extract_points(addresses, da)

In [ ]:
buildings.to_parquet('s3://carbonplan-risks/ocr/v0/intermediates/buildings_risk.parquet')
addresses.to_parquet('s3://carbonplan-risks/ocr/v0/intermediates/addresses_risk.parquet')